# Create rotations chart data

# Setup

In [32]:
import pandas as pd
import numpy as np
import nba_api

In [68]:
from nba_api.stats.static.teams import get_teams, find_teams_by_nickname
from nba_api.stats.endpoints import LeagueGameFinder
from nba_api.stats.endpoints import PlayByPlayV2
from nba_api.stats.endpoints import PlayByPlay
from nba_api.stats.endpoints import BoxScorePlayerTrackV2

In [34]:
spurs_info = find_teams_by_nickname('Spurs')
spurs_id = spurs_info[0]['id']

# find all Spurs games
gamefinder = LeagueGameFinder(team_id_nullable=spurs_id,
                                              season_nullable='2019-20')
team_box_scores_df = gamefinder.get_data_frames()[0]
team_box_scores_df.head(n=10)

spurs_games = list(team_box_scores_df.GAME_ID.unique()) # grab unique list of game IDs

test_game = spurs_games[0]

In [95]:
# get list of rosters and starting players
roster_data_df = BoxScorePlayerTrackV2(game_id=test_game).data_sets[0].get_data_frame()
starters_df = roster_data_df[roster_data_df.START_POSITION.isin(['F', 'G', 'C'])]
spurs_starters = list(starters_df[starters_df.TEAM_ABBREVIATION == 'SAS'].PLAYER_NAME)
opp_starters = list(starters_df[starters_df.TEAM_ABBREVIATION != 'SAS'].PLAYER_NAME)
spurs_roster = list(roster_data_df[roster_data_df.TEAM_ABBREVIATION == 'SAS'].PLAYER_NAME)
opp_roster = list(roster_data_df[roster_data_df.TEAM_ABBREVIATION != 'SAS'].PLAYER_NAME)

In [35]:
pbp = PlayByPlayV2(game_id=test_game)

In [36]:
pbp_df = pbp[0].get_data_frame()
pbp_df.head(n=10)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0021901314,2,12,0,1,6:38 PM,12:00,None,None,None,...,None,None,0,0,None,NaN,None,None,None,0
1,0021901314,4,10,0,1,6:38 PM,12:00,Jump Ball Bradley vs. Poeltl: Tip to Ingles,None,None,...,Spurs,SAS,4,204060,Joe Ingles,1.610613e+09,Utah,Jazz,UTA,1
2,0021901314,7,1,47,1,6:38 PM,11:46,Clarkson 7' Turnaround Jump Shot (2 PTS) (Mitc...,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,1
3,0021901314,9,2,78,1,6:38 PM,11:31,O'Neale BLOCK (1 BLK),None,MISS Samanic 4' Floating Jump Shot,...,None,None,4,1626220,Royce O'Neale,1.610613e+09,Utah,Jazz,UTA,1
4,0021901314,11,4,0,1,6:38 PM,11:28,Bradley REBOUND (Off:0 Def:1),None,None,...,None,None,0,0,None,NaN,None,None,None,1
5,0021901314,12,2,1,1,6:38 PM,11:23,MISS O'Neale 25' 3PT Jump Shot,None,None,...,None,None,0,0,None,NaN,None,None,None,1
6,0021901314,13,4,0,1,6:38 PM,11:17,None,None,Samanic REBOUND (Off:0 Def:1),...,None,None,0,0,None,NaN,None,None,None,1
7,0021901314,14,2,78,1,6:38 PM,11:12,None,None,MISS Walker IV 14' Floating Jump Shot,...,None,None,0,0,None,NaN,None,None,None,1
8,0021901314,15,4,0,1,6:38 PM,11:10,Bradley REBOUND (Off:0 Def:2),None,None,...,None,None,0,0,None,NaN,None,None,None,1
9,0021901314,16,2,1,1,6:39 PM,10:58,MISS Clarkson 25' 3PT Jump Shot,None,None,...,None,None,0,0,None,NaN,None,None,None,1


# Functions

In [38]:
def conv_time_to_sec(time):
    split_time = time.split(':')
    new_time = int(split_time[0]) * 60 + int(split_time[1])
    new_time = 12*60 - new_time
    return new_time

In [39]:
def parse_player_in(pbp_text):
    split_text = pbp_text.split('FOR')
    
    return (split_text[0][4:].strip())

In [40]:
def parse_player_out(pbp_text):
    split_text = pbp_text.split('FOR')
    
    return (split_text[1].strip())

# Filtering & Exploration

In [41]:
pbp_subs = pbp_df[pbp_df['HOMEDESCRIPTION'].str.contains('SUB') == True].copy()
pbp_subs.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
45,0021901314,67,8,0,1,6:49 PM,6:16,SUB: Niang FOR Clarkson,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,0
46,0021901314,68,8,0,1,6:49 PM,6:16,SUB: Oni FOR Ingles,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,0
67,0021901314,101,8,0,1,6:57 PM,3:51,SUB: Ingles FOR Bradley,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,0
68,0021901314,102,8,0,1,6:57 PM,3:51,SUB: Davis FOR O'Neale,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,0
69,0021901314,103,8,0,1,6:57 PM,3:51,SUB: Clarkson FOR Mitchell,None,None,...,Jazz,UTA,0,0,None,NaN,None,None,None,0


In [53]:
pbp_df[pbp_df['PERIOD'] == 2]

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
121,0021901314,174,12,0,2,7:10 PM,12:00,None,None,None,...,None,None,0,0,None,NaN,None,None,None,0
122,0021901314,175,2,79,2,7:10 PM,11:49,None,None,MISS Belinelli 21' Pullup Jump Shot,...,None,None,0,0,None,NaN,None,None,None,1
123,0021901314,176,4,0,2,7:10 PM,11:46,JAZZ Rebound,None,None,...,None,None,0,0,None,NaN,None,None,None,0
124,0021901314,178,2,6,2,7:10 PM,11:33,MISS Davis 4' Driving Layup,None,Eubanks BLOCK (2 BLK),...,None,None,5,1629234,Drew Eubanks,1.610613e+09,San Antonio,Spurs,SAS,1
125,0021901314,180,4,0,2,7:10 PM,11:32,Davis REBOUND (Off:3 Def:1),None,None,...,None,None,0,0,None,NaN,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,0021901314,337,2,107,2,7:36 PM,0:12,MISS Bradley 1' Tip Dunk Shot,None,None,...,None,None,0,0,None,NaN,None,None,None,1
237,0021901314,338,4,0,2,7:36 PM,0:10,None,None,Murray REBOUND (Off:1 Def:6),...,None,None,0,0,None,NaN,None,None,None,1
238,0021901314,339,2,1,2,7:36 PM,0:01,None,None,MISS Samanic 27' 3PT Jump Shot,...,None,None,0,0,None,NaN,None,None,None,1
239,0021901314,340,4,0,2,7:36 PM,0:01,None,None,Spurs Rebound,...,None,None,0,0,None,NaN,None,None,None,0


In [42]:
pbp_subs.columns

Index(['GAME_ID', 'EVENTNUM', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'PERIOD',
       'WCTIMESTRING', 'PCTIMESTRING', 'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION',
       'VISITORDESCRIPTION', 'SCORE', 'SCOREMARGIN', 'PERSON1TYPE',
       'PLAYER1_ID', 'PLAYER1_NAME', 'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_CITY',
       'PLAYER1_TEAM_NICKNAME', 'PLAYER1_TEAM_ABBREVIATION', 'PERSON2TYPE',
       'PLAYER2_ID', 'PLAYER2_NAME', 'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_CITY',
       'PLAYER2_TEAM_NICKNAME', 'PLAYER2_TEAM_ABBREVIATION', 'PERSON3TYPE',
       'PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY',
       'PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION',
       'VIDEO_AVAILABLE_FLAG'],
      dtype='object')

In [44]:
pbp_subs['PCTIMESTRING'].dtypes

dtype('O')

In [45]:
conv_time_to_sec(pbp_subs['PCTIMESTRING'].iloc[0])

344

In [61]:
# convert time to seconds

pbp_subs['period_time'] = pbp_subs.PCTIMESTRING.apply(conv_time_to_sec)

# convert period time to total time
# period - 1 to account for extra 12 mins
pbp_subs['game_time'] = pbp_subs['period_time'] + ((pbp_subs['PERIOD']-1) * (12*60))

In [67]:
pbp_subs.head(n=15)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,period_time,game_time
45,0021901314,67,8,0,1,6:49 PM,6:16,SUB: Niang FOR Clarkson,None,None,...,0,0,None,NaN,None,None,None,0,344,344
46,0021901314,68,8,0,1,6:49 PM,6:16,SUB: Oni FOR Ingles,None,None,...,0,0,None,NaN,None,None,None,0,344,344
67,0021901314,101,8,0,1,6:57 PM,3:51,SUB: Ingles FOR Bradley,None,None,...,0,0,None,NaN,None,None,None,0,489,489
68,0021901314,102,8,0,1,6:57 PM,3:51,SUB: Davis FOR O'Neale,None,None,...,0,0,None,NaN,None,None,None,0,489,489
69,0021901314,103,8,0,1,6:57 PM,3:51,SUB: Clarkson FOR Mitchell,None,None,...,0,0,None,NaN,None,None,None,0,489,489
70,0021901314,104,8,0,1,6:57 PM,3:51,SUB: Morgan FOR Oni,None,None,...,0,0,None,NaN,None,None,None,0,489,489
140,0021901314,202,8,0,2,7:13 PM,9:15,SUB: Mitchell FOR Ingles,None,None,...,0,0,None,NaN,None,None,None,0,165,885
141,0021901314,203,8,0,2,7:13 PM,9:15,SUB: O'Neale FOR Morgan,None,None,...,0,0,None,NaN,None,None,None,0,165,885
142,0021901314,204,8,0,2,7:13 PM,9:15,SUB: Oni FOR Niang,None,None,...,0,0,None,NaN,None,None,None,0,165,885
155,0021901314,223,8,0,2,7:15 PM,8:24,SUB: Brantley FOR Clarkson,None,None,...,0,0,None,NaN,None,None,None,0,216,936


In [63]:
filtered_pbp_subs = pbp_subs[['PERIOD', 'game_time', 'HOMEDESCRIPTION']].copy()

In [64]:
# create subs chart
filtered_pbp_subs['player_in'] = filtered_pbp_subs['HOMEDESCRIPTION'].apply(parse_player_in)
filtered_pbp_subs['player_out'] = filtered_pbp_subs['HOMEDESCRIPTION'].apply(parse_player_out)

In [65]:
subs_out_df = filtered_pbp_subs[['PERIOD', 'game_time', 'player_in', 'player_out']].copy()
subs_out_df

,PERIOD,game_time,player_in,player_out
45,1,344,Niang,Clarkson
46,1,344,Oni,Ingles
67,1,489,Ingles,Bradley
68,1,489,Davis,O'Neale
69,1,489,Clarkson,Mitchell
70,1,489,Morgan,Oni
140,2,885,Mitchell,Ingles
141,2,885,O'Neale,Morgan
142,2,885,Oni,Niang
155,2,936,Brantley,Clarkson


In [51]:
subs_out_df.to_json('data/subs_data.json', orient='records')

# Create rotations data for visualization

In [ ]:
# find starters for each team
# at the start of each quarter, track down the five players initally in the game